* 분기별, 월별 예금은행 대출금리는 잘 나와있어서 생략합니다.
* 전처리 할 때 read_csv, to_csv 이런 거 할 때 encoding="CP949" 인코딩 옵션 넣어줘야 한글 안 깨져요
* read_csv에 CP949 안 먹히면 UTF8로 해주세요 (아마)

In [1]:
# 전처리를 위한 라이브러리 불러오기
import pandas as pd
import numpy as np

In [2]:
# GDP는 아주 단순합니다
# transpose()로 컬럼 세로로 만든 다음
# 필요한 부분만 iloc 이용해서 추출해서 내보내면 끝

gdp = pd.read_csv("../raw_data/국내총생산.csv", encoding="UTF8").transpose()
gdp_real = gdp.iloc[4:].rename(columns={0:"GDP"})
gdp_real.to_csv("../전처리/GDP.csv", encoding="CP949")

In [3]:
# 소비자물가지수 데이터도 단순합니다
# transpose()로 컬럼 세로로 만들어서
# 필요한 부분을 추출하고 내보내면 끝

sobija = pd.read_csv("../raw_data/소비자물가지수.csv").transpose()
sobija_real = sobija.iloc[5:].rename(columns={0:"소비자물가지수"})
sobija_real.to_csv("../전처리/소비자물가지수.csv", encoding="CP949")

In [4]:
# 주택종합 매매가격지수, 전세가격지수 전처리

# 매매가격지수 데이터 가져오기
data = pd.read_excel("../raw_data/주택종합 매매가격지수_20220808.xlsx")

# 데이터 확인 후 필요한 부분만 추출해 회전하고 합치기
data_s = data[0:1].transpose().reset_index().drop(index=0)

# 데이터 확인 (필요없으면 주석처리)
data_s

,index,0
1,1986-01-01 09:00:00,27.682
2,1986-02-01 09:00:00,27.682
3,1986-03-01 09:00:00,27.724
4,1986-04-01 09:00:00,27.516
5,1986-05-01 09:00:00,27.392
...,...,...
435,2022-03-01 09:00:00,100.306
436,2022-04-01 09:00:00,100.517
437,2022-05-01 09:00:00,100.768
438,2022-06-01 09:00:00,100.869


In [5]:
# 시계열 처리 수행
# 시계열 데이터로 전환
data_s['index'] = pd.to_datetime(data_s['index'], format="%Y-%m-%d %HH:%M:%S").dt.strftime("%Y-%m")

In [6]:
# CSV로 뽑아줍시다
# 컬럼 이름 바꿔주는 rename에 값 바꿔주세요
data_s.rename(columns={0:"주택종합 매매가격지수"}).reset_index(drop=True).to_csv("../전처리/주택종합_매매가격지수.csv",encoding='CP949')

In [7]:
# 전세가격 지수는 똑같으니까 한번에 갑니다

data2 = pd.read_excel("../raw_data/주택종합 전세가격지수_20220808.xlsx")
data2_s = data2[0:1].transpose().reset_index().drop(index=0)
data2_s['index'] = pd.to_datetime(data2_s['index'], format="%Y-%m-%d %HH:%M:%S").dt.strftime("%Y-%m")
data2_s.rename(columns={0:"주택종합 전세가격지수"}).reset_index(drop=True).to_csv("../전처리/주택종합_전세가격지수.csv",encoding='CP949')

In [8]:
# 실업률 가봅시다 -> 실업자 위주로 갑시다
job = pd.read_csv("../raw_data/연령별_경제활동인구_총괄_20220808152350(월별).csv", encoding='CP949').transpose()

# 임시로 데이터를 담아놓을 데이터프레임과 리스트 선언
job_test = pd.DataFrame()
job_test_real = []

# 반복문으로 돌릴건데
# 가져온 csv 파일 끝까지 돌릴 거고
for i in range(len(job)):
    # 맨 끝에 도달하면 탈출합니다
    if(i*8-4 == len(job)+3): break
    # 맨 처음 한 번은 건너 뜁니다 (우리가 원하는 데이터 값이 아니라서)
    if(i==0): continue

    # 실업률만 골라서 저장하고 있습니다
    job_test = job.iloc[i*8-4]
    # 실업률을 리스트에 저장하고 있습니다
    job_test_real.append(job_test)

# 불필요한 컬럼을 삭제하고, 컬럼의 이름을 바꿔서 내보내봅시다
job_test_real_df = pd.DataFrame(job_test_real).drop(columns=[0]).rename(columns={1:"15세 이상", 2:"15-19", 3:"20-29", 4:"30-39", 5:"40-49", 6:"50-59", 7:"60세 이상", 8:"15-64", 9:"15-24", 10:"15-29"})

# 실업자로 구하고 싶으면 i*8-4, len(job)+3 으로 변경해주세요
# 실업률로 구하고 싶으면 i*8-1, len(job)+6 으로 변경해주세요
# iloc로 필요한 성분을 조회해서 그 때 그 때 위치에 맞춰서 조정하면 다른 데이터도 뽑을 수 있을 거예요

In [9]:
# index에 대한 시계열 처리 후 저장까지 갑시다

job_test_real_df = job_test_real_df.reset_index()
job_test_real_df['index'] = pd.to_datetime(job_test_real_df['index'], format="%Y.%m.").dt.strftime("%Y-%m")
job_test_real_df.set_index('index').to_csv("../전처리/실업자_전처리(천명).csv", encoding="CP949")

# 밑에 파일 이름 바꾸는 것도 잊지마세요 -> 괜히 덮어쓰기 처리 되면 돌리는 사람만 억울함

In [11]:
# 코스피 지수 추출
import pandas_datareader as pdd
import datetime

start = datetime.datetime(2004,1,1)
end = datetime.datetime(2022,7,31)
df_null = pdd.DataReader("^KS11", "yahoo", start, end)
df = df_null.dropna()

df.to_csv("../전처리/kospi.csv", encoding="CP949")

In [12]:
# 코스피200 지수 추출
# 코스피 전체 지수는 야후 파이넨스에 잘만 있는데 코스피200은 부실하네요...

# 코스피200에 대한 데이터는 따로 찾아주세요
k_200 = pd.read_csv("../raw_data/종합주가지수_200.csv", encoding="CP949")

# 시계열 처리
k_200['일자'] = pd.to_datetime(k_200['일자'], format="%Y/%m/%d").dt.strftime("%Y-%m-%d")

# 데이터가 최신순으로 나와있어서 재정렬 들어갑니다
k_200 = k_200.sort_values('일자', ascending=True).reset_index(drop=True)
k_200.to_csv("../전처리/kospi200.csv", encoding='CP949')